In [ ]:
%%capture
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore') 
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
from pandas_datareader import data as pdr

In [ ]:
url = "https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2"
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = "https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2"
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3809    0  3809    0     0   9570      0 --:--:-- --:--:-- --:--:--  9570
100  503k  100  503k    0     0   485k      0  0:00:01  0:00:01 --:--:-- 4597k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3845    0  3845    0     0  15078      0 --:--:-- --:--:-- --:--:-- 15078
100  406k  100  406k    0     0   429k      0 --:--:-- --:--:-- --:--:-- 2326k


In [ ]:
import talib

In [ ]:
nse_Symbol=['ADANIPORTS.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJAJFINSV.NS',
 'BAJFINANCE.NS',
 'BHARTIARTL.NS',
 'BPCL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFC.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'IOC.NS',
 'ITC.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SBIN.NS',
 'SHREECEM.NS',
 'SUNPHARMA.NS',
 'TATACONSUM.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TCS.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'UPL.NS',
 'WIPRO.NS']
 

In [ ]:
# stock='RELIANCE.NS'

In [ ]:
def checker(stock):
  monthly_date='2019-01-01'
  weekly_date='2021-01-01'
  daily_date='2021-05-01'
  last_date='2021-09-20'
  df=pd.DataFrame()
  df['date'] = pd.date_range(start='05/01/2021',end='09/21/2021', freq='D')
  # def checker(stock_name,start_date,end_date):
  df1 = yf.download(stock, 
                    start=monthly_date, 
                    end=last_date,
                    interval='1mo')
  df1.reset_index(inplace=True)
  df1=df1.dropna()
  df1=df1.reset_index().drop(['index','Open', 'High', 'Low', 'Adj Close', 'Volume'],axis=1)
  df1['RSI'] = talib.RSI(df1['Close'], timeperiod=14) #RSI
  # df1=df1[-1:]
  df1['RSI']=df1['RSI'].shift(1)
  # def checker(stock_name,start_date,end_date):
  df2 = yf.download(stock, 
                    start=weekly_date, 
                    end=last_date,
                    interval='1wk')
  df2.reset_index(inplace=True)
  df2=df2.dropna()
  df2=df2.reset_index().drop(['index','Open', 'High', 'Low', 'Adj Close', 'Volume'],axis=1)
  df2['RSI'] = talib.RSI(df2['Close'], timeperiod=14) #RSI
  df2['RSI']=df2['RSI'].shift(1)
  # df1=df1[-1:]
  ###
  # def checker(stock_name,start_date,end_date):
  df3 = yf.download(stock, 
                    start=daily_date, 
                    end=last_date,
                    interval='1D')
  df3.reset_index(inplace=True)
  df3=df3.dropna()
  df3=df3.reset_index().drop(['index','Open', 'High', 'Low', 'Adj Close', 'Volume'],axis=1)
  df3['RSI'] = talib.RSI(df3['Close'], timeperiod=14) #RSI
  # df1=df1[-1:]
  df_i=pd.merge(df,df3,how='left',left_on='date',right_on='Date')
  df4=pd.merge(df_i,df2,how='left',on='Date')
  df4=df4.fillna(method='ffill')
  df5=pd.merge(df4,df1,how='left',on='Date')
  df5=df5.fillna(method='ffill')
  ##
  df6=df5[df5['RSI'] > 60]
  df6=df6[df6['RSI_y'] > 60]
  df6=df6[df6['RSI_x'] < 65]
  df6=df6[df6['RSI_x'] > 55]
  ##
  df6=df6.drop_duplicates(subset='Date')
  # print(df6.head(20))
  return (list(df6['Close']),list(df6['Date']))


In [ ]:
%%capture
 
# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nse_Symbol):
    selected_stocks[i]=checker(i)
final_selection=pd.DataFrame.from_dict(selected_stocks,orient='index',columns=['Close','Dates'])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={'index':'Ticker'})
final_selection=final_selection.reset_index().drop('index',axis=1)
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection['Dates'][i])):
    new_entry.append([final_selection['Ticker'][i],final_selection['Dates'][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=['Stock','Date'])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({'Stock': j[0],'Date':j[1]}, ignore_index=True)
    except:
      pass
# print('completed, please check dataframe "final" ')
final=final.dropna().reset_index(drop=True)
final['Date']=final['Date'].astype('str')

In [ ]:
final.head(20)

,Stock,Date
0,ADANIPORTS.NS,2021-06-01
1,ADANIPORTS.NS,2021-06-02
2,ADANIPORTS.NS,2021-06-10
3,ADANIPORTS.NS,2021-06-11
4,ASIANPAINT.NS,2021-06-22
5,ASIANPAINT.NS,2021-06-23
6,ASIANPAINT.NS,2021-06-25
7,ASIANPAINT.NS,2021-06-28
8,ASIANPAINT.NS,2021-06-29
9,ASIANPAINT.NS,2021-06-30
